In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="d4data/biomedical-ner-all")

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [1]:
import os
import json
split = "train.json"
data = json.load(open(f"data/{split}"))
files = os.listdir("data/CT json/")
#files.remove(".DS_Store")

In [2]:
files_data = {file[:-5]:json.load(open(f"data/CT json/{file}")) for file in files}

In [3]:
import pandas as pd
from collections import defaultdict as dd
df = pd.DataFrame(data).T
df

,Type,Section_id,Primary_id,Secondary_id,Statement,Label
5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction
86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction
dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment
20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction
f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction
...,...,...,...,...,...,...
f37774f4-db96-4aa6-b3a1-626953faeecf,Comparison,Eligibility,NCT00213980,NCT02536339,"Adequate blood, kidney, and hepatic function a...",Entailment
4fef4cdf-53bf-4239-9d31-4710fd3edc6f,Single,Results,NCT01605396,NaN,The Ridaforolimus + Dalotuzumab + Exemestane g...,Contradiction
331affb2-f8e9-4a55-ac4c-62d2ecc4f80b,Single,Intervention,NCT00181363,NaN,The only difference between the interventions ...,Entailment
a577e819-c928-4217-8743-f4809e852919,Single,Eligibility,NCT00834678,NaN,Patients must have a white blood cell count ab...,Entailment


In [4]:
data_dict = {}

for index, row in df.iterrows():
    key = row['Statement']
    value = row.drop('Statement').to_dict()
    data_dict[f"{key}_____{row['Label']}"] = value
    

In [5]:
len(data_dict)

1700

In [48]:
hypotheses = list(df[df['Label'] == 'Entailment']['Statement'])

In [49]:
len(hypotheses)

850

In [50]:
from types import SimpleNamespace

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

def convert_to_desired_format(entity_list, text):
    # Extracting text from the provided entities
    #text = " ".join([entity['word'] for entity in entity_list])

    # Combining consecutive entities with the same type
    combined_entities = []
    current_entity = None

    for entity in entity_list:
        if entity['score'] < 0.7:
            continue
        if current_entity is None:
            current_entity = {"start": entity['start'], "end": entity['end'], "label": entity['entity'].split('-')[-1]}
        elif entity['entity'].split('-')[-1] == current_entity['label'] and entity['start'] in [current_entity['end'],current_entity['end']+1]:
            current_entity['end'] = entity['end']
        else:
            combined_entities.append(current_entity)
            current_entity = {"start": entity['start'], "end": entity['end'], 
                              "label": entity['entity'].split('-')[-1],
                            }

    if current_entity is not None:
        combined_entities.append(current_entity)

    # Creating the final output in the desired format
    output = {"text": text, "ents": combined_entities, "title": None}
    return output




In [51]:
from tqdm import tqdm
res = []
outs = []
for text in tqdm(hypotheses):
    ents = pipe(text)
    
    res.append(convert_to_desired_format(ents, text))
    

100%|█████████████████████████████████████████████████████████████████████████████████████████| 850/850 [00:18<00:00, 45.40it/s]


In [55]:
displacy.render(res[19], manual=True, style='ent', jupyter=True)

In [52]:
outs=[]
for out in res:
    ents = out['ents']
    for ent in ents:
        ent['text'] = out['text'] 
        ent['entity'] = out['text'][ent['start']:ent['end']]
    outs.extend(ents)

## Entities in Hypotheses

In [53]:
import pandas as pd
entities = pd.DataFrame(outs)

In [54]:
entities

,start,end,label,text,entity
0,0,13,Detailed_description,Heart-related adverse events were recorded in ...,Heart-related
1,14,28,Sign_symptom,Heart-related adverse events were recorded in ...,adverse events
2,12,23,Age,Adele is an 85 year old woman with Stage II hi...,85 year old
3,24,29,Sex,Adele is an 85 year old woman with Stage II hi...,woman
4,35,43,Detailed_description,Adele is an 85 year old woman with Stage II hi...,Stage II
...,...,...,...,...,...
2316,137,154,Diagnostic_procedure,The most common adverse event in the primary t...,Ejection fraction
2317,14,26,Disease_disorder,Patients with cancer cells in lymph nodes abov...,cancer cells
2318,30,62,Biological_structure,Patients with cancer cells in lymph nodes abov...,lymph nodes above the collarbone
2319,21,43,Diagnostic_procedure,Patients must have a white blood cell count ab...,white blood cell count


In [15]:
entities.loc[0]['text'][59:69]

'capecitabi'

In [55]:
entities_no_duplicate = entities.drop_duplicates()

In [56]:
entities_no_duplicate

,start,end,label,text,entity
0,0,13,Detailed_description,Heart-related adverse events were recorded in ...,Heart-related
1,14,28,Sign_symptom,Heart-related adverse events were recorded in ...,adverse events
2,12,23,Age,Adele is an 85 year old woman with Stage II hi...,85 year old
3,24,29,Sex,Adele is an 85 year old woman with Stage II hi...,woman
4,35,43,Detailed_description,Adele is an 85 year old woman with Stage II hi...,Stage II
...,...,...,...,...,...
2316,137,154,Diagnostic_procedure,The most common adverse event in the primary t...,Ejection fraction
2317,14,26,Disease_disorder,Patients with cancer cells in lymph nodes abov...,cancer cells
2318,30,62,Biological_structure,Patients with cancer cells in lymph nodes abov...,lymph nodes above the collarbone
2319,21,43,Diagnostic_procedure,Patients must have a white blood cell count ab...,white blood cell count


In [57]:
entities_no_duplicate['label'].unique()

array(['Detailed_description', 'Sign_symptom', 'Age', 'Sex',
       'Disease_disorder', 'Biological_structure', 'Diagnostic_procedure',
       'Therapeutic_procedure', 'Medication', 'Lab_value',
       'Administration', 'Coreference', 'Duration', 'Severity', 'Date',
       'Subject', 'History', 'Distance', 'Frequency',
       'Personal_background', 'Dosage', 'Clinical_event',
       'Quantitative_concept', 'Outcome', 'Activity', 'Family_history',
       'Volume'], dtype=object)

## Handling Numerical Data

In [58]:
# Age column
set(entities[entities['label'] == 'Age']['entity'])

{'32 year old',
 '34 years',
 '51-73 years',
 '55 year old',
 '56 year old',
 '57 year old',
 '72 year old',
 '75 year old',
 '85 year old',
 'age',
 'age of 20',
 'between 18-25',
 'older',
 'over',
 'over the age of 18'}

In [20]:
list(entities[(entities['label'] == 'Age') & (entities['entity'] == 'below')]['text'])

['Completely disabled patients below the age of 20, totally confined to bed or chair and unable to carry on any selfcare are eligible for the primary trial but excluded from the secondary trial.']

In [21]:
all_rows = []

In [59]:
import re

def process_age(text):
    match = re.search(r'(\d+) year old', text)
    if match:
        age = int(match.group(1))
        new_age = age + 10
        return re.sub(r'\d+ year old', f'{new_age} year old', text)
    return text

old2new = dd(str)
for text in hypotheses:
    match = re.search(r'(\d+) year old', text)
    if match:
        age = int(match.group(1))
        new_age = age + 10
        old2new[text] = re.sub(r'\d+ year old', f'{new_age} year old', text)

In [60]:
len(old2new)

7

In [61]:
old2new

defaultdict(str,
            {'Adele is an 85 year old woman with Stage II histologically confirmed ER+ breast cancer with an ECOG of 0, she is eligible for the primary trial': 'Adele is an 95 year old woman with Stage II histologically confirmed ER+ breast cancer with an ECOG of 0, she is eligible for the primary trial',
             'Ae-Cha is a 32 year old Korean woman with an inoperable breast cancer, she is ineligible for both the secondary trial and the primary trial.': 'Ae-Cha is a 42 year old Korean woman with an inoperable breast cancer, she is ineligible for both the secondary trial and the primary trial.',
             'A 55 year old postmenopausal patient with sarcoidosis would be excluded from the primary trial as it would prevent prevent prolonged follow-up.': 'A 65 year old postmenopausal patient with sarcoidosis would be excluded from the primary trial as it would prevent prevent prolonged follow-up.',
             'A 72 year old patient suffering from dementia would be

In [194]:
new_data = []
for h, h_data in data_dict.items():
    h, label = h.split('_____')

    new_rw = dd()
    new_rw.update(h_data)
    new_rw['Statement'] = h
    new_data.append(new_rw)
    if h in old2new:
        h_new = old2new[h]
        new_rw = dd()
        new_rw.update(h_data)
        new_rw['Statement'] = h_new
        if h_data['Label'] == 'Contradiction':
            new_rw['Label'] = 'Entailment'
        if h_data['Label'] == 'Entailment':
            new_rw['Label'] = 'Contradiction'

        new_data.append(new_rw)

In [195]:
all_rows.extend(new_data)

In [196]:
pd.DataFrame(new_data).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
3,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
4,Comparison,Intervention,NCT00852930,NCT02308020,Contradiction,Laser Therapy is in each cohort of the primary...
...,...,...,...,...,...,...
1710,Comparison,Eligibility,NCT00213980,NCT02536339,Entailment,"Adequate blood, kidney, and hepatic function a..."
1711,Single,Results,NCT01605396,NaN,Contradiction,The Ridaforolimus + Dalotuzumab + Exemestane g...
1712,Single,Intervention,NCT00181363,NaN,Entailment,The only difference between the interventions ...
1713,Single,Eligibility,NCT00834678,NaN,Entailment,Patients must have a white blood cell count ab...


In [62]:
# Handling Other Age samples
map_ = {
    'below': 'above',
    'older': 'younger',
    'over': 'under', 
}
opp_map = {}
for k, v in map_.items():
    opp_map[k] = v
    opp_map[v] = k

In [63]:
old2new = dd(str)
for text in hypotheses:
    new_text = text
    to_replace = []
    for word, replacement in opp_map.items():
        if f' {word} ' in text:
            to_replace.append(word)
    if len(to_replace) == 0:
        continue
    for word in to_replace:
        new_text = new_text.replace(word, opp_map[word])

    old2new[text] = new_text


In [64]:
len(old2new)

45

In [65]:
old2new

defaultdict(str,
            {'The shortest PFS in the Talazoparib group of the primary trial was over a month shorter than the Median PFS for that group.': 'The shortest PFS in the Talazoparib group of the primary trial was under a month shorter than the Median PFS for that group.',
             'Participants of the primary trial must be older than 18, have histologically confirmed stage 4 breast cancer, ECOG<2 and a life expectancy exceeding 6 months.': 'Participants of the primary trial must be younger than 18, have histologically confirmed stage 4 breast cancer, ECOG<2 and a life expectancy exceeding 6 months.',
             'At least one participant in the primary trial receiving an oral treatment of film-coated Afatinib tablet at a starting dose of 40 milligram (mg) once daily and weekly 10 minutes intravenous infusions of Vinorelbine 25 mg/meter^2 (meter=m) had a PFS over 9 months.': 'At least one participant in the primary trial receiving an oral treatment of film-coated Afatin

In [200]:
new_data = []
for h, h_data in data_dict.items():
    h, label = h.split('_____')

    new_rw = dd()
    new_rw.update(h_data)
    new_rw['Statement'] = h
    new_data.append(new_rw)
    if h in old2new:
        h_new = old2new[h]
        new_rw = dd()
        new_rw.update(h_data)
        new_rw['Statement'] = h_new
        if h_data['Label'] == 'Contradiction':
            new_rw['Label'] = 'Entailment'
        if h_data['Label'] == 'Entailment':
            new_rw['Label'] = 'Contradiction'

        new_data.append(new_rw)

In [201]:
all_rows.extend(new_data)

In [202]:
pd.DataFrame(new_data).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Single,Eligibility,NCT00662129,NaN,Entailment,"Patients with Platelet count under 100,000/mm¬..."
3,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
4,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
...,...,...,...,...,...,...
1790,Single,Results,NCT01605396,NaN,Entailment,The Ridaforolimus + Dalotuzumab + Exemestane g...
1791,Single,Intervention,NCT00181363,NaN,Entailment,The only difference between the interventions ...
1792,Single,Eligibility,NCT00834678,NaN,Entailment,Patients must have a white blood cell count ab...
1793,Single,Eligibility,NCT00834678,NaN,Contradiction,Patients must have a white blood cell count be...


In [76]:
entities_no_duplicate['label'].unique()

array(['Administration', 'Medication', 'Diagnostic_procedure',
       'Lab_value', 'Detailed_description', 'Sign_symptom',
       'Biological_structure', 'Disease_disorder',
       'Therapeutic_procedure', 'Age', 'Sex', 'Dosage', 'Duration',
       'Coreference', 'Severity', 'History', 'Personal_background',
       'Date', 'Subject', 'Distance', 'Frequency', 'Clinical_event',
       'Outcome', 'Quantitative_concept', 'Activity', 'Family_history',
       'Nonbiological_location', 'Volume'], dtype=object)

In [77]:
# Lab Value column
set(entities[entities['label'] == 'Lab_value']['entity'])

{'%',
 '+ve',
 '.',
 '.5xUL',
 '/percentage',
 '0',
 '0%',
 '0.0015%',
 '0.98 cm3 increase',
 '1',
 '1 more',
 '1,700/mm¬¨‚â',
 '1,733/mm',
 '1.35',
 '1.5',
 '1/42',
 '1/7',
 '10',
 '10% higher',
 '10% more',
 '10.0',
 '10.3% higher',
 '100',
 '100,000/mm¬¨‚',
 '100,000/¬¨¬µl',
 '11',
 '11.1%',
 '11.5',
 '12',
 '120',
 '125',
 '13% lower',
 '14',
 '15',
 '15% less',
 '1500',
 '16',
 '16, 17 or 20 hours',
 '16.67% more',
 '16/179',
 '175',
 '18',
 '18% lower',
 '19.57%',
 '1cm3',
 '2',
 '2 more',
 '2 negative',
 '2 positive',
 '2.38% more',
 '2.7months',
 '2.87 cm3 decrease',
 '2/680',
 '2/93',
 '20',
 '20%',
 '20% difference',
 '20/20',
 '23',
 '25%',
 '25% more',
 '250 mg/m^2',
 '27.6%',
 '3',
 '3 mg/ks',
 '3 more',
 '3/112',
 '3/4',
 '30',
 '30%',
 '30% decrease',
 '32',
 '33% higher',
 '38% more',
 '3A',
 '3M',
 '3M1',
 '4',
 '4 million units/m^2',
 '4 more',
 '4% more',
 '40 mg/m^2',
 '5',
 '5 million units/m',
 '5 more',
 '5%',
 '5% less',
 '5,000/mm3',
 '50 x 109/l',
 '50,0000 x 

In [66]:
map_ = {
    'increase': 'decrease',
    'increased': 'decreased',
    'higher': 'lower',
    'high': 'low',
    'more' : 'less',
    'negative': 'positive',
    'at least': 'at most',
    'increasing': 'decreasing',
    'larger': 'smaller',
    'large': 'small',
    'maximum': 'minimum',
    'short': 'long',
    'shorter': 'longer',
    
}
opp_map = {}
for k, v in map_.items():
    opp_map[k] = v
    opp_map[v] = k

In [67]:
old2new = dd(str)
for text in hypotheses:
    new_text = text
    to_replace = []
    for word, replacement in opp_map.items():
        if f' {word} ' in text:
            to_replace.append(word)
    if len(to_replace) == 0:
        continue
    for word in to_replace:
        new_text = new_text.replace(word, opp_map[word])
            
    old2new[text] = new_text

In [68]:
len(old2new)

183

In [69]:
old2new

defaultdict(str,
            {'The shortest PFS in the Talazoparib group of the primary trial was over a month shorter than the Median PFS for that group.': 'The shortest PFS in the Talazoparib group of the primary trial was over a month longer than the Median PFS for that group.',
             'Most patients in the Letrozole group of the primary trial had a decreased Bone Mineral Density of the Lumbar Spine after 24 months.': 'Most patients in the Letrozole group of the primary trial had a increased Bone Mineral Density of the Lumbar Spine after 24 months.',
             'One patient in the primary trial had abnormally low red blood cells, white blood cells, and platelets.': 'One patient in the primary trial had abnormally high red blood cells, white blood cells, and platelets.',
             'In total, across both cohorts of the primary trial, there were at least 2 patients with a fever.': 'In total, across both cohorts of the primary trial, there were at most 2 patients with a fever

In [206]:
new_data = []
for h, h_data in data_dict.items():
    h, label = h.split('_____')

    new_rw = dd()
    new_rw.update(h_data)
    new_rw['Statement'] = h
    new_data.append(new_rw)
    if h in old2new:
        h_new = old2new[h]
        new_rw = dd()
        new_rw.update(h_data)
        new_rw['Statement'] = h_new
        if h_data['Label'] == 'Contradiction':
            new_rw['Label'] = 'Entailment'
        if h_data['Label'] == 'Entailment':
            new_rw['Label'] = 'Contradiction'

        new_data.append(new_rw)

In [207]:
all_rows.extend(new_data)

In [208]:
pd.DataFrame(new_data).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
3,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
4,Comparison,Intervention,NCT00852930,NCT02308020,Contradiction,Laser Therapy is in each cohort of the primary...
...,...,...,...,...,...,...
2087,Comparison,Eligibility,NCT00213980,NCT02536339,Entailment,"Adequate blood, kidney, and hepatic function a..."
2088,Single,Results,NCT01605396,NaN,Contradiction,The Ridaforolimus + Dalotuzumab + Exemestane g...
2089,Single,Intervention,NCT00181363,NaN,Entailment,The only difference between the interventions ...
2090,Single,Eligibility,NCT00834678,NaN,Entailment,Patients must have a white blood cell count ab...


In [94]:
entities_no_duplicate['label'].unique()

array(['Administration', 'Medication', 'Diagnostic_procedure',
       'Lab_value', 'Detailed_description', 'Sign_symptom',
       'Biological_structure', 'Disease_disorder',
       'Therapeutic_procedure', 'Age', 'Sex', 'Dosage', 'Duration',
       'Coreference', 'Severity', 'History', 'Personal_background',
       'Date', 'Subject', 'Distance', 'Frequency', 'Clinical_event',
       'Outcome', 'Quantitative_concept', 'Activity', 'Family_history',
       'Nonbiological_location', 'Volume'], dtype=object)

In [95]:
# Lab Value column
set(entities[entities['label'] == 'Dosage']['entity'])

{'0.05',
 '1',
 '1 milligram',
 '10 minutes',
 '100 mg',
 '1000 mg PO daily',
 '1000mg daily',
 '120 mg',
 '12mg Q',
 '145 mg/m¬¨',
 '150',
 '150 mg/Day',
 '1500 mg',
 '1500 mg PO',
 '1500 milligrams',
 '150mg',
 '150mg once every 4 weeks',
 '160 mg',
 '2',
 '2 weeks',
 '200 milligrams (',
 '200mg',
 '21 day cycles',
 '25 mg/meter^2',
 '250',
 '250 mg',
 '250mg',
 '28',
 '28 day',
 '2mg/kg',
 '3 doses',
 '3 ml Perflut',
 '3 times per',
 '3 times per week',
 '3.25 mg',
 '30',
 '30 mg',
 '300 mg',
 '30mg',
 '4',
 '4 mg',
 '4 mg Delayed',
 '4 mg/',
 '40 mg',
 '40 mg/m2',
 '40 mg/m^2',
 '40 milligram',
 '40 milligrams',
 '40.5 Gy',
 '400 mg/day',
 '40mg PO',
 '4mg/kg',
 '5 doses',
 '5 mg group',
 '50 mg',
 '500mg',
 '50mg',
 '50mg/',
 '50mg/m2',
 '6',
 '6 mg',
 '6 tablets',
 '60 mg',
 '600 mg',
 '600 mg/m2',
 '70',
 '750mg',
 '75mg/m2',
 '800',
 '800mg',
 '80mg/m2',
 'High',
 'High Dose',
 'Low',
 'Low Dose',
 'QD',
 'Twice Daily',
 'cycles',
 'daily',
 'daily dose',
 'dose',
 'doses',
 'd

In [104]:
list(entities[(entities['label'] == 'Dosage') & (entities['entity'] == '6 tablets')]['text'])

['the primary trial participants must take 6 tablets of Lapatinib Monotherapy (1500g) PO once daily.',
 'the primary trial participants must take 6 tablets of Lapatinib Monotherapy (250mg) PO once daily.']

In [70]:
values_to_perturb = {
    '1 milligram': '10 milligrams',
    '10 minutes': '20 minutes',
    '100 mg': '200 mg',
    '1000 mg PO daily': '2000 mg PO daily',
    '150 mg/Day': '300 mg/Day',
    '1500 mg': '3000 mg',
    '150mg': '300mg',
    '150mg once every 4 weeks': '300mg once every 4 weeks',
    '2 weeks': '3 weeks',
    '200 milligrams (': '400 milligrams (',
    '200mg': '400mg',
    '21 day cycles': '42 day cycles',
    '25 mg/meter^2': '50 mg/meter^2',
    '250': '500',
    '250 mg': '500 mg',
    '250mg': '500mg',
    '28 day': '15 day',
    '2mg/kg': '3mg/kg',
    '3 doses': '4 doses',
    '3 ml Perflut': '4 ml Perflut',
    '3 times per': '4 times per',
    '3 times per week': '4 times per week',
    '3.25 mg': '6.5 mg',
    '4 mg Delayed': '8 mg Delayed',
    '4 mg/': '8 mg/',
    '40 mg': '80 mg',
    '40 mg/m2': '80 mg/m2',
    '40 mg/m^2': '80 mg/m^2',
    '40 milligram': '80 milligram',
    '40 milligrams': '80 milligrams',
    '40.5 Gy': '81 Gy',
    '400 mg/day': '800 mg/day',
    '40mg PO': '80mg PO',
    '4mg/kg': '8mg/kg',
    '5 doses': '6 doses',
    '5 mg group': '10 mg group',
    '50 mg': '100 mg',
    '500mg': '1000mg',
    '50mg': '100mg',
    '50mg/': '100mg/',
    '50mg/m2': '100mg/m2',
    '6 mg': '7 mg',
    '6 tablets': '8 tablets',
    '60 mg': '80 mg',
    '600 mg': '800 mg',
    '600 mg/m2': '1200 mg/m2',
    '750mg': '1500mg',
    '75mg/m2': '150mg/m2',
    '800mg': '900mg',
    '80mg/m2': '90mg/m2',
    'once': 'twice',
}

In [71]:
old2new = dd(str)
for text in hypotheses:
    new_text = text
    to_replace = []
    for word, replacement in values_to_perturb.items():
        if f' {word} ' in text:
            to_replace.append(word)
    if len(to_replace) == 0:
        continue
    for word in to_replace:
        new_text = new_text.replace(word, values_to_perturb[word])
            
    old2new[text] = new_text

In [72]:
len(old2new)

33

In [73]:
old2new

defaultdict(str,
            {'At least one participant in the primary trial receiving an oral treatment of film-coated Afatinib tablet at a starting dose of 40 milligram (mg) once daily and weekly 10 minutes intravenous infusions of Vinorelbine 25 mg/meter^2 (meter=m) had a PFS over 9 months.': 'At least one participant in the primary trial receiving an oral treatment of film-coated Afatinib tablet at a starting dose of 80 milligram (mg) twice daily and weekly 20 minutes intravenous infusions of Vinorelbine 50 mg/meter^2 (meter=m) had a PFS over 9 months.',
             'A patient who underwent T-cell transfer therapy in the past 2 weeks would be excluded from the primary trial.': 'A patient who underwent T-cell transfer therapy in the past 3 weeks would be excluded from the primary trial.',
             'Intervention 1 of the primary trial require participants to take 300 mg alpelisib once daily.': 'Intervention 1 of the primary trial require participants to take 300 mg alpelisib twi

In [212]:
new_data = []
for h, h_data in data_dict.items():
    h, label = h.split('_____')

    new_rw = dd()
    new_rw.update(h_data)
    new_rw['Statement'] = h
    new_data.append(new_rw)
    if h in old2new:
        h_new = old2new[h]
        new_rw = dd()
        new_rw.update(h_data)
        new_rw['Statement'] = h_new
        if h_data['Label'] == 'Contradiction':
            new_rw['Label'] = 'Entailment'
        if h_data['Label'] == 'Entailment':
            new_rw['Label'] = 'Contradiction'

        new_data.append(new_rw)

In [213]:
all_rows.extend(new_data)

In [214]:
pd.DataFrame(new_data).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
3,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
4,Comparison,Intervention,NCT00852930,NCT02308020,Contradiction,Laser Therapy is in each cohort of the primary...
...,...,...,...,...,...,...
1766,Comparison,Eligibility,NCT00213980,NCT02536339,Entailment,"Adequate blood, kidney, and hepatic function a..."
1767,Single,Results,NCT01605396,NaN,Contradiction,The Ridaforolimus + Dalotuzumab + Exemestane g...
1768,Single,Intervention,NCT00181363,NaN,Entailment,The only difference between the interventions ...
1769,Single,Eligibility,NCT00834678,NaN,Entailment,Patients must have a white blood cell count ab...


In [115]:
entities_no_duplicate['label'].unique()

array(['Administration', 'Medication', 'Diagnostic_procedure',
       'Lab_value', 'Detailed_description', 'Sign_symptom',
       'Biological_structure', 'Disease_disorder',
       'Therapeutic_procedure', 'Age', 'Sex', 'Dosage', 'Duration',
       'Coreference', 'Severity', 'History', 'Personal_background',
       'Date', 'Subject', 'Distance', 'Frequency', 'Clinical_event',
       'Outcome', 'Quantitative_concept', 'Activity', 'Family_history',
       'Nonbiological_location', 'Volume'], dtype=object)

In [116]:
# Lab Value column
set(entities[entities['label'] == 'Duration']['entity'])

{'1',
 '11 months',
 '12 weeks',
 '16 Weeks',
 '17',
 '18 months',
 '2',
 '2 - 6 months',
 '2 years',
 '2.5 months',
 '200 days',
 '21 day',
 '28',
 '3',
 '3 months',
 '3 years',
 '5',
 '5 year',
 '6',
 '6 months',
 'a',
 'a month',
 'last 6 months',
 'less',
 'months',
 'more',
 'more than 7',
 'more than a',
 'more than a year',
 'next 5',
 'number of days',
 'over 14 weeks',
 'over 21 days',
 'over 36 hours',
 'over 5 years',
 'over a',
 'over a month',
 'over a year',
 'over half a year',
 'past 2 weeks',
 'shorter period',
 'time',
 'two weeks',
 'under a year',
 'weeks',
 'within',
 'year',
 'years'}

In [74]:
map_ = {
    'minute': 'hour',
    'hour': 'day',
    'day': 'week',
    'week': 'month',
    'month': 'year',
    
}

old2new = dd(str)
for text in hypotheses:
    new_text = text
    to_replace = []
    for word, replacement in map_.items():
        if f' {word}' in text.lower():
            to_replace.append(word)
    if len(to_replace) == 0:
        continue
    for word in to_replace:
        new_text = new_text.replace(word, map_[word])
        #new_text = re.sub(r'\b' + re.escape(word) + r'\b', values_to_perturb[word], new_text, flags=re.IGNORECASE)
            
    old2new[text] = new_text

In [75]:
old2new

defaultdict(str,
            {'Agatha had her 50th birthday last week, she has a histologically confirmed adenocarcinoma of the breast, with no evidence of metastatic disease. She is eligible for the primary trial but not the secondary trial.': 'Agatha had her 50th birthday last month, she has a histologically confirmed adenocarcinoma of the breast, with no evidence of metastatic disease. She is eligible for the primary trial but not the secondary trial.',
             'The shortest PFS in the Talazoparib group of the primary trial was over a month shorter than the Median PFS for that group.': 'The shortest PFS in the Talazoparib group of the primary trial was over a year shorter than the Median PFS for that group.',
             'Most patients in the Letrozole group of the primary trial had a decreased Bone Mineral Density of the Lumbar Spine after 24 months.': 'Most patients in the Letrozole group of the primary trial had a decreased Bone Mineral Density of the Lumbar Spine after 24 

In [217]:
new_data = []
for h, h_data in data_dict.items():
    h, label = h.split('_____')

    new_rw = dd()
    new_rw.update(h_data)
    new_rw['Statement'] = h
    new_data.append(new_rw)
    if h in old2new:
        h_new = old2new[h]
        new_rw = dd()
        new_rw.update(h_data)
        new_rw['Statement'] = h_new
        if h_data['Label'] == 'Contradiction':
            new_rw['Label'] = 'Entailment'
        if h_data['Label'] == 'Entailment':
            new_rw['Label'] = 'Contradiction'

        new_data.append(new_rw)

In [218]:
all_rows.extend(new_data)

In [219]:
pd.DataFrame(new_data).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
3,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
4,Comparison,Intervention,NCT00852930,NCT02308020,Contradiction,Laser Therapy is in each cohort of the primary...
...,...,...,...,...,...,...
1882,Single,Results,NCT01605396,NaN,Contradiction,The Ridaforolimus + Dalotuzumab + Exemestane g...
1883,Single,Results,NCT01605396,NaN,Entailment,The Ridaforolimus + Dalotuzumab + Exemestane g...
1884,Single,Intervention,NCT00181363,NaN,Entailment,The only difference between the interventions ...
1885,Single,Eligibility,NCT00834678,NaN,Entailment,Patients must have a white blood cell count ab...


In [139]:
entities_no_duplicate['label'].unique()

array(['Administration', 'Medication', 'Diagnostic_procedure',
       'Lab_value', 'Detailed_description', 'Sign_symptom',
       'Biological_structure', 'Disease_disorder',
       'Therapeutic_procedure', 'Age', 'Sex', 'Dosage', 'Duration',
       'Coreference', 'Severity', 'History', 'Personal_background',
       'Date', 'Subject', 'Distance', 'Frequency', 'Clinical_event',
       'Outcome', 'Quantitative_concept', 'Activity', 'Family_history',
       'Nonbiological_location', 'Volume'], dtype=object)

In [144]:
# Lab Value column
set(entities[entities['label'] == 'Volume']['entity'])

{'237 ml'}

In [223]:
pd.DataFrame(all_rows).drop_duplicates()

,Type,Section_id,Primary_id,Secondary_id,Label,Statement
0,Comparison,Intervention,NCT01928186,NCT00684983,Contradiction,All the primary trial participants do not rece...
1,Single,Eligibility,NCT00662129,NaN,Contradiction,"Patients with Platelet count over 100,000/mm¬¨..."
2,Comparison,Adverse Events,NCT00093145,NCT00703326,Entailment,Heart-related adverse events were recorded in ...
3,Single,Eligibility,NCT01097642,NaN,Contradiction,Adult Patients with histologic confirmation of...
4,Comparison,Intervention,NCT00852930,NCT02308020,Contradiction,Laser Therapy is in each cohort of the primary...
...,...,...,...,...,...,...
9218,Single,Results,NCT00558272,NaN,Entailment,The biggest change in Change From Baseline in ...
9224,Single,Eligibility,NCT01094184,NaN,Entailment,A patient who had a Joint injection in the las...
9236,Single,Results,NCT00240071,NaN,Entailment,At least one participant of the primary trial ...
9250,Single,Results,NCT00708214,NaN,Entailment,All Participants with disease progression (Aft...


In [225]:
pd.DataFrame(all_rows).drop_duplicates().to_json('dataset_5.json', orient='index')

In [76]:
to_verify = {
    'Adele is an 85 year old woman with Stage II histologically confirmed ER+ breast cancer with an ECOG of 0, she is eligible for the primary trial': 'Adele is an 95 year old woman with Stage II histologically confirmed ER+ breast cancer with an ECOG of 0, she is eligible for the primary trial',
    'The shortest PFS in the Talazoparib group of the primary trial was over a month shorter than the Median PFS for that group.': 'The shortest PFS in the Talazoparib group of the primary trial was under a month shorter than the Median PFS for that group.',
    'Most patients in the Letrozole group of the primary trial had a decreased Bone Mineral Density of the Lumbar Spine after 24 months.': 'Most patients in the Letrozole group of the primary trial had a increased Bone Mineral Density of the Lumbar Spine after 24 months.',
    'A patient who underwent T-cell transfer therapy in the past 2 weeks would be excluded from the primary trial.': 'A patient who underwent T-cell transfer therapy in the past 3 weeks would be excluded from the primary trial.',
    'One patient in the primary trial had a 2.87 cm3 decrease in Total MRI Functional Tumor Volume (FTV) over 3 months.': 'One patient in the primary trial had a 2.87 cm3 decrease in Total MRI Functional Tumor Volume (FTV) over 3 years.',

}

In [77]:
#to_verify = {h.lower():p for h, p in to_verify.items()}
avail = list(to_verify.keys())

In [81]:
data_expanded = []
for _id, value in data.items():
    statement = value["Statement"]
    if statement in avail:
        temp = {}
        temp["id"] = _id
        p_nctid = value["Primary_id"]
        s_nctid = value.get("Secondary_id")
        section_id = value["Section_id"]

        primary_evidence = files_data[p_nctid][section_id]
        temp["statement"] = statement
        sentence = 'Primary Evidence: ' + ''.join(primary_evidence)
        temp["label"] = value["Label"]

        if s_nctid is not None:
            secondary_evidence = files_data[s_nctid][section_id]
            sentence += 'Secondary Evidence' + ''.join(secondary_evidence)
        temp['premise'] = sentence
        temp['perturbation'] = to_verify[statement]
        data_expanded.append(temp)

In [82]:
data_expanded

[{'id': '8765009d-ffc4-4395-ab7a-11ecdfd43a40',
  'statement': 'Adele is an 85 year old woman with Stage II histologically confirmed ER+ breast cancer with an ECOG of 0, she is eligible for the primary trial',
  'label': 'Entailment',
  'premise': "Primary Evidence: Inclusion Criteria:  Must be female with histologically confirmed breast cancer  Stage II-IV disease  ER and/or PR positive  ECOG Performance Status 0-1  Tumor must be present following core needle biopsy as determined by physical exam or radiographic evaluation.  Postmenopausal  No prior treatment for current breast cancer. No other active malignancy is allowed.Adequately treated basal cell, squamous cell skin cancer, in situ cervical cancer, or any other cancer from which the patient has been disease-free for 5 years is permitted. Biphosphonates and palliative radiation for bone metastasis is permitted while on study.  Hormone replacement therapy must be discontinued. It is not permitted during the time on study.Exclusion

In [84]:
json.dump({'samples': data_expanded}, open('numerical_perturbations.json', 'w+'), indent=4)